In [3]:
search_for = 125
start_from = 0
threads = 10
thread = 8
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004053831100463867
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 35
all cases: 320327931
type: [1, 1, 1, 1, 125] 125
cases of this type: 244140625
100000 14.530210702260257
200000 23.827336711173622
300000 23.56719727484284
400000 24.9704659550966
500000 26.356517297467448
600000 24.628076658859136
700000 23.868563659895035
800000 24.69126376089465
900000 23.62875189925692
1000000 11.01831775094311
1100000 23.583482032190567
1200000 23.68290309023731
1300000 24.218942474718578
1400000 22.776554119518785
1500000 23.3823282211202
1600000 23.29328837934924
1700000 23.80231018139309
1800000 23.86303630032712
1900000 23.578348482152148
2000000 7.459998803177428
2100000 25.234223978359964
2200000 29.419220160893
2300000 27.66203336075747
2400000 28.96041476023784
2500000 27.663740323659425
2600000 27.83680467054785
2700000 26.233260365448274
2800000 27.133035576124154
2900000 25.96030601417656
3000000 11.5782050

29400000 23.771387965700036
29500000 23.502780807401393
29600000 9.984755717110643
29700000 23.55870205098873
29800000 24.087307092567784
29900000 21.75359385084836
30000000 22.099161363968808
30100000 21.45758212836387
30200000 19.940623929923078
30300000 9.944105950788948
30400000 20.336731573976227
30500000 20.752652081752018
30600000 21.88071238010565
30700000 21.435394044768273
30800000 23.37211566461022
30900000 23.4750507442273
31000000 18.63956898050435
31100000 15.560192629511468
31200000 23.463669085125645
31300000 23.104044405330626
31400000 23.413195694729094
31500000 18.41777906029148
31600000 15.484336584780458
31700000 23.204451970204914
31800000 22.70021907738555
31900000 24.386232928671046
32000000 21.577002319606855
32100000 20.372449440355368
32200000 19.403069113126822
32300000 10.428053249495978
32400000 20.71771260310732
32500000 20.00294816897254
32600000 21.389235654998387
32700000 22.52920195529057
32800000 22.82552909208719
32900000 23.671608262134594
33000000

59100000 9.813126928110876
59200000 18.535686278820016
59300000 19.663599439080727
59400000 19.147055067046825
59500000 18.494225201190336
59600000 8.603429709186615
59700000 18.3541314980752
59800000 19.837826647532744
59900000 17.56751940410768
60000000 19.001815388342113
60100000 10.260892031110613
60200000 19.89108004114567
60300000 21.834045875847853
60400000 21.541924099858477
60500000 21.25116030191307
60600000 21.937614295032834
60700000 20.98086629073508
60800000 20.289906155038885
60900000 19.606305758155976
61000000 19.63156737722232
61100000 8.359913224787439
61200000 18.2433277073248
61300000 18.234609523626467
61400000 18.080556151269096
61500000 16.144336602823426
61600000 9.131084633060354
61700000 17.864154893471216
61800000 19.60709121203476
61900000 17.575293294258056
62000000 14.786464778009544
62100000 13.293801654046444
62200000 19.023696915533776
62300000 19.727829736681393
62400000 19.866997342566776
62500000 20.054675668916513
62600000 19.64142414532943
6270000

88900000 6.309493162498249
89000000 16.451347806406133
89100000 16.26864936530849
89200000 8.471331866300131
89300000 16.19211421006328
89400000 15.98437115859013
89500000 15.4601214196725
89600000 17.19650494930231
89700000 18.129047267211735
89800000 17.746400757709722
89900000 18.163533454462435
90000000 18.414575042269703
90100000 18.40180111386497
90200000 16.459246634453997
90300000 16.471031378379518
90400000 16.00149443743346
90500000 15.420303861966762
90600000 8.079287173779226
90700000 15.374915478613419
90800000 14.19733911154651
90900000 8.372361554947584
91000000 15.526059066570873
91100000 8.781307735545802
91200000 15.718620895589094
91300000 16.41087078673619
91400000 15.26334423513812
91500000 16.569670692826207
91600000 16.774781498772686
91700000 18.33191471713733
91800000 17.258999521365904
91900000 16.978962202024324
92000000 17.04959203189451
92100000 16.13772736525726
92200000 14.90592442568713
92300000 15.429993377817226
92400000 15.94469628844365
92500000 15.8

117900000 1.82343479916416
118000000 1.7464505396927557
118100000 1.5885917583756215
118200000 1.5236441083061278
118300000 1.5866980814975127
118400000 1.67489073822315
118500000 1.5567564374656815
118600000 1.670705746360543
118700000 1.5964933405912287
118800000 1.543965317861372
118900000 1.5551202523590153
119000000 1.623941200574251
119100000 1.8038815663215817
119200000 1.5784385217873533
119300000 1.5679272454925213
119400000 1.609589347972361
119500000 1.6393445313617276
119600000 1.6424174389216895
119700000 1.5920106939113539
119800000 1.6432909808171716
119900000 1.6899074759110835
120000000 1.6250819534061314
120100000 1.7745362280545096
120200000 1.5961601075170846
120300000 1.5493101944543104
120400000 1.6430532190505318
120500000 1.6131591643209333
120600000 1.5195051865167202
120700000 1.594733266133178
120800000 1.6018507839159906
120900000 1.6621170860829353
121000000 1.5365730499600119
121100000 1.6533165468294846
121200000 1.474276385035799
121300000 1.505263055768

...

204300000 8.0579281376394
204400000 8.264878359215487
204500000 9.631748182994633
204600000 9.198549272958411
204700000 9.083055459236897
204800000 4.177678680492493
204900000 9.559097914145925
205000000 8.95362531733543
205100000 9.226524438076217
205200000 8.82042187365231
205300000 9.666660736876832
205400000 5.942353863678855
205500000 8.146856774490947
205600000 8.844954925357163
205700000 8.131407209939384
205800000 8.028098291656217
205900000 7.72794348687036
206000000 7.890977145834272
206100000 2.9515693564957126
206200000 7.950384623895062
206300000 7.89503648551384
206400000 7.9454399968357095
206500000 8.374690851849799
206600000 8.418850615295279
206700000 8.162727311573478
206800000 4.739158565548828
206900000 8.578984624718952
207000000 8.796028589217695
207100000 8.610355160680463
207200000 8.7082920460986
207300000 7.71352007339865
207400000 5.2946204735574165
207500000 9.013025309189544
207600000 8.224812594874189
207700000 8.192788701948608
207800000 7.85204163557354

233600000 6.5081711795988095
233700000 6.534458128997478
233800000 6.424988960177533
233900000 6.357822810404942
234000000 6.842170481207604
234100000 6.545055936509818
234200000 6.748744013487027
234300000 4.552559516305762
234400000 5.435628451677429
234500000 3.291322897251232
234600000 6.587971466465796
234700000 6.6361693820421
234800000 6.632380811750864
234900000 6.6802278716266414
235000000 6.410217490938792
235100000 6.458672937228631
235200000 6.566710764696396
235300000 6.682617675015712
235400000 2.27211949520512
235500000 6.957523975917779
235600000 6.506471079428698
235700000 6.564808756344376
235800000 6.649940782430985
235900000 6.934785115802538
236000000 6.570656664766294
236100000 6.70358768328545
236200000 6.598207417451872
236300000 2.739517135789384
236400000 4.317678916828398
236500000 5.519609984711169
236600000 6.290436679761033
236700000 6.176013139410461
236800000 6.217912709643734
236900000 6.160000855125712
237000000 5.771254686881107
237100000 5.7216453744

262200000 1.631826999601531
262300000 1.5051426330015703
262400000 1.4073037377994915
262500000 1.8223734606206619
262600000 1.120339078720869
262700000 0.48657231016270913
262800000 0.48292958658121626
262900000 1.0071836322056875
263000000 1.9462843779258614
263100000 1.4342065025371313
263200000 1.5862574467436015
263300000 1.6082465645901058
263400000 1.4829265718735734
263500000 1.5375270903925116
263600000 1.5427296453256847
263700000 1.7946406637845138
263800000 1.644941688730594
263900000 1.5850444088689204
264000000 1.5250748488726793
264100000 1.4743872004438334
264200000 1.4546029218555951
264300000 1.507504259847655
264400000 1.5852992843629274
264500000 1.3654590803953222
264600000 0.7835637511222602
264700000 0.46470558141397234
264800000 0.620305341709443
264900000 1.318803111896515
265000000 1.698546272877878
265100000 1.505741423642328
265200000 1.4114034142750382
265300000 1.4608576149395716
265400000 1.509144254380354
265500000 1.4897670325102965
265600000 1.50564047

290700000 0.8363594718462667
290800000 0.8650454208169123
290900000 0.8037876275128697
291000000 0.9251634692795873
291100000 0.9292064731933171
291200000 0.8863597678024285
291300000 0.7728540938260993
291400000 0.8425428986463016
291500000 0.8355795747960666
291600000 0.7668191865957955
291700000 0.6984508099439721
291800000 0.8925105625318037
291900000 0.5507906849173009
292000000 0.23459214569201536
292100000 0.23475608291257355
292200000 0.49938279597129825
292300000 0.8742748083828549
292400000 0.6866747508408533
292500000 0.801536158257711
292600000 0.8128255695679374
292700000 0.7377516258427389
292800000 0.7936299160958291
292900000 0.7818964813517293
best so far: 0
type: [1, 1, 1, 25, 5] 125
cases of this type: 9765625
293000000 0.8917304195389265
293100000 0.7984018900906881
293200000 0.7419619349803541
293300000 0.7582354439545003
293400000 0.7364335942283313
293500000 0.629321539629923
293600000 0.5734957080175439
293700000 0.5942854864547252
293800000 0.6871289214500785
2

best so far: 0
type: [1, 1, 25, 5, 1] 125
cases of this type: 78125
317200000 0.022911548415817155
best so far: 0
type: [1, 1, 125, 1, 1] 125
cases of this type: 15625
best so far: 0
type: [1, 5, 1, 1, 25] 125
cases of this type: 1953125
317300000 0.02918168154730797
317400000 0.0641353406201621
317500000 0.05248825610607399
317600000 0.052030999261759386
317700000 0.04945762755023043
317800000 0.04660590045301385
317900000 0.04383404284394251
318000000 0.04957037782046795
318100000 0.03456006300466259
318200000 0.015624791714945767
318300000 0.014979719894061487
318400000 0.014520844086772204
318500000 0.0247923731155422
318600000 0.035833694053033986
318700000 0.03194948949133291
318800000 0.029023730751377343
318900000 0.025586740943974257
319000000 0.024781720406056116
319100000 0.02239434695660273
319200000 0.022240716718544565
best so far: 0
type: [1, 5, 1, 5, 5] 125
cases of this type: 390625
319300000 0.014922232639068366
319400000 0.008456207396880785
319500000 0.0073433029927